In [4]:
import torch 
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:

import torch
import torch.nn as nn
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from matplotlib import cm


# 模型搭建
class Net(nn.Module):
    def __init__(self, NN): # NL n个l（线性，全连接）隐藏层， NN 输入数据的维数， 128 256
        # NL是有多少层隐藏层
        # NN是每层的神经元数量
        super(Net, self).__init__()

        self.input_layer = nn.Linear(2, NN)
        self.hidden_layer1 = nn.Linear(NN,int(NN/2)) ## 原文这里用NN，我这里用的下采样，经过实验验证，“等采样”更优
        self.hidden_layer2 = nn.Linear(int(NN/2), int(NN/2))  ## 原文这里用NN，我这里用的下采样，经过实验验证，“等采样”更优
        self.output_layer = nn.Linear(int(NN/2), 1)

    def forward(self, x): # 一种特殊的方法 __call__() 回调
        out = torch.tanh(self.input_layer(x))
        out = torch.tanh(self.hidden_layer1(out))
        out = torch.tanh(self.hidden_layer2(out))
        out_final = self.output_layer(out)
        return out_final

In [16]:
def pde(x, net):
    u = net(x)
    u_tx = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(net(x)),
                               create_graph=True, allow_unused=True)[0]  # 求偏导数
    u_t = u_tx[:, 0].unsqueeze(-1)
    u_x = u_tx[:, 1].unsqueeze(-1)
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x),
                               create_graph=True, allow_unused=True)[0][:,1].unsqueeze(-1)  # 求偏导数

    H = torch.tensor([[1.2, 0.8], [-0.6, 0.9]],dtype=torch.float32)
    M = torch.tensor([[0.5,0.7], [0.3,1.0]],dtype=torch.float32)
    sigma = torch.tensor([[0.8],[1.1]],dtype=torch.float32) 
    C = torch.tensor([[1.6, 0.0], [0.0, 1.1]],dtype=torch.float32)  # Positive semi-definite
    D = torch.tensor([[0.5, 0.0], [0.0, 0.7]],dtype=torch.float32)  # Positive definite
    R = torch.tensor([[0.9, 0.0], [0.0, 1.0]],dtype=torch.float32)  # Positive semi-definite
    alpha = torch.tensor([[1], [1]],dtype =torch.float32)
    alphat = torch.tensor([1,1],dtype=torch.float32)
    realx = x[:, 1:]

    return     u_t.squeeze() + 0.5 * (sigma @ sigma.transpose(0,1) @ u_xx).transpose(0,1).squeeze() + u_x.transpose(0,1) @ H @ realx + u_x.transpose(0,1) @ M @ alpha + realx.transpose(0,1) @ C @ realx + alphat @ D @ alpha



In [17]:
net = Net(30)
mse_cost_function = torch.nn.MSELoss(reduction='mean')  # Mean squared error
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

t_bc_zeros = np.zeros((2, 1))
x_in_pos_one = np.ones((2, 1))
x_in_neg_one = -np.ones((2, 1))
u_in_zeros = np.zeros((2, 1))

R = np.array([[0.9, 0.0], [0.0, 1.0]] )

iterations = 10000
for epoch in range(iterations):
    optimizer.zero_grad()  # 梯度归0

    t_in_var = np.random.uniform(low=0, high=1.0, size=(2, 1))
    x_bc_var = np.random.uniform(low=-1.0, high=1.0, size=(2, 1))
    u_bc_sin = x_bc_var.transpose() @ R @ x_bc_var

    pt_x_bc_var = Variable(torch.from_numpy(x_bc_var).float(), requires_grad=False)
    pt_t_bc_zeros = Variable(torch.from_numpy(t_bc_zeros).float(), requires_grad=False)
    pt_u_bc_sin = Variable(torch.from_numpy(u_bc_sin).float(), requires_grad=False)
    pt_x_in_pos_one = Variable(torch.from_numpy(x_in_pos_one).float(), requires_grad=False)
    pt_x_in_neg_one = Variable(torch.from_numpy(x_in_neg_one).float(), requires_grad=False)
    pt_t_in_var = Variable(torch.from_numpy(t_in_var).float(), requires_grad=False)
    pt_u_in_zeros = Variable(torch.from_numpy(u_in_zeros).float(), requires_grad=False)

    net_bc_out = net(torch.cat([pt_t_bc_zeros, pt_x_bc_var], 1))  # u(x,t)的输出
    mse_u_2 = mse_cost_function(net_bc_out, pt_u_bc_sin)  # e = u(x,t)-(-sin(pi*x))  公式（2）


    x_collocation = np.random.uniform(low=-1.0, high=1.0, size=(2, 1))
    t_collocation = np.random.uniform(low=0.0, high=1.0, size=(2, 1))
    all_zeros = np.zeros((2000, 1))
    pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True)
    pt_t_collocation = Variable(torch.from_numpy(t_collocation).float(), requires_grad=True)
    pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=False)

     # 确保pt_t_collocation和pt_x_collocation有相同的数据类型
    pt_t_collocation = pt_t_collocation.to(torch.float32)
    pt_x_collocation = pt_x_collocation.to(torch.float32)

    # 将变量x,t带入公式（1）
    f_out = pde(torch.cat([pt_t_collocation, pt_x_collocation], 1), net)  # output of f(x,t) 公式（1）
    mse_f_1 = mse_cost_function(f_out, pt_all_zeros)

        # 将误差(损失)累加起来
    loss = mse_f_1 + mse_u_2 

    loss.backward()  # 反向传播
    optimizer.step()  # This is equivalent to : theta_new = theta_old - alpha * derivative of J w.r.t theta

    with torch.autograd.no_grad():
        if epoch % 100 == 0:
            print(epoch, "Traning Loss:", loss.data)




/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2000, 1])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0 Traning Loss: tensor(5.2837)
100 Traning Loss: tensor(6.4104)
200 Traning Loss: tensor(1.9076)
300 Traning Loss: tensor(3.1256)
400 Traning Loss: tensor(0.0700)
500 Traning Loss: tensor(0.0623)
600 Traning Loss: tensor(1.6365)
700 Traning Loss: tensor(0.6291)
800 Traning Loss: tensor(0.7593)
900 Traning Loss: tensor(0.2838)
1000 Traning Loss: tensor(1.7971)
1100 Traning Loss: tensor(0.3762)
1200 Traning Loss: tensor(0.0904)
1300 Traning Loss: tensor(0.5712)
1400 Traning Loss: tensor(2.3536)
1500 Traning Loss: tensor(0.8637)
1600 Traning Loss: tensor(1.2382)
1700 Traning Loss: tensor(0.0590)
1800 Traning Loss: tensor(0.4951)
1900 Traning Loss: tensor(1.3279)
2000 Traning Loss: tensor(0.7408)
2100 Traning Loss: tensor(1.7497)
2200 Traning Loss: tensor(0.4769)
2300 Traning Loss: tensor(0.2073)
2400 Traning Loss: tensor(0.2072)
2500 Traning Loss: tensor(0.1078)
2600 Traning Loss: tensor(0.9364)
2700 Traning Loss: tensor(0.3927)
2800 Traning Loss: tensor(1.1364)
2900 Traning Loss: tensor(